# Scrapping prices data

## Getting the source page

Example ticker is HPQ, with page data <https://www.optionseducation.org/quotes.html?quote=HPQ>. This page has been selected since it's well protected agaisnt robots (they don´t like you put your nose on them...)

This page offers undelying and options prices along with other information.

First focus on the stock price. it can be found in a table below the ticker and market information. If the page is opened with Chrome and then right click on the price, selecting "Inspeccionar", a new window will be opened showing the html associated to the price information, in this case the data price is embedded in a table with the following structure:
<table cellpadding="2" cellspacing="1" border="0" bgcolor="#999999" width="100%">
<tbody><tr bgcolor="#cccccc" align="center" class="s1">
<td>Price</td><td>Change&nbsp;(%)</td><td>52&nbsp;wk&nbsp;High</td><td>52&nbsp;wk&nbsp;Low</td><td>Stock Volume&nbsp;
					<a href="javascript:openHelp(14)" alt="Open Help"><img src="/design/images/ico/question_btn.gif" width="10" height="10" border="0" alt="Open Help"></a></td><td>Avg. options volume&nbsp;
					<a href="javascript:openHelp(10222)" alt="Open Help"><img src="/design/images/ico/question_btn.gif" width="10" height="10" border="0" alt="Open Help"></a></td><td>Avg. options open interest&nbsp;
					<a href="javascript:openHelp(11)" alt="Open Help"><img src="/design/images/ico/question_btn.gif" width="10" height="10" border="0" alt="Open Help"></a></td>
</tr>
<tr bgcolor="#eeeeee" align="center" class="s1">
<td>21.91</td><td><nobr><img src="/design/images/ico/arrow_bottom.gif" alt="-" border="0" align="absmiddle" width="7" height="9">&nbsp; 
						-0.01&nbsp;(-0.05%)
					</nobr></td><td><nobr>24.65&nbsp;09-Mar</nobr></td><td><nobr>17.19&nbsp;06-Jul</nobr></td><td>7683571</td><td>9,150</td><td>192,050</td>
</tr>
</tbody></table>
```
<table cellpadding="2" cellspacing="1" border="0" bgcolor="#999999" width="100%">
    <tbody>
        <tr bgcolor="#cccccc" align="center" class="s1">
            <td>Price</td>
            ...
        </tr>
        <tr bgcolor="#eeeeee" align="center" class="s1">
            <td>21.91</td>
            ...
        </tr>
    </tbody>
</table>
```
A way of identifying this table in the structure is to look for the "Price" string as data of first row and then we know the actual price will be the data of the second row.

Let's get the web page and save to a file:


In [5]:
import requests
from bs4 import BeautifulSoup
 
req = requests.get('https://www.optionseducation.org/quotes.html?quote=HPQ')
soup_oic = BeautifulSoup(req.text, "lxml")
with open("hpq_oic","w", encoding="UTF-8") as fp:
    fp.write(req.text)

If we open the hpq.html file we will realize that this table is not indeed in the web page!!.

This has happened because the web page is just a loaded page as is indicated by the tag before:

```
<iframe  src="https://oic.ivolatility.com/oic_adv_options.j?cnt=a4fe7416d6719b97decda2b53ef5ac290da77083b4582616&ticker=HPQ" width="100%" height="100%" style="height:800px" ></iframe>
```

If we open that web page 
<https://oic.ivolatility.com/oic_adv_options.j?cnt=a4fe7416d6719b97decda2b53ef5ac290da77083b4582616&ticker=HPQ>
we can see that just the table is shown

So, this page is the one that we have to collect:


In [6]:
req = requests.get('https://oic.ivolatility.com/oic_adv_options.j?cnt=a4fe7416d6719b97decda2b53ef5ac290da77083b4582616&ticker=HPQ')
soup_table = BeautifulSoup(req.text, "lxml")
with open("hpq_table.html","w", encoding="UTF-8") as fp:
    fp.write(req.text)

the only issue is the address has metadata embedded, the only way to access to this last page is first access to the optionseducation.org quote page, get the link to the volatility.com page and access it

So to get the address information, we must loop over all iframe tags and look for the one which contents the "ivolatiliy" string inside the src attribute:

In [13]:
for iframe in soup_oic.find_all('iframe'):
    if 'ivolatility' in iframe['src']:
        address = iframe['src']
        print(address)

https://oic.ivolatility.com/oic_adv_options.j?cnt=a4fe7416d6719b9707a161a4aa4ba08e0ed7c2e7d264f4a0&ticker=HPQ


And now we can access to the page

In [14]:
req = requests.get(address)
soup_table = BeautifulSoup(req.text, "lxml")
with open("hpq_table","w", encoding="UTF-8") as fp:
    fp.write(req.text)

Let's repeat the process but now with another ticker, example ON:


In [21]:
import requests
from bs4 import BeautifulSoup

ticker = 'ON'
url = 'https://www.optionseducation.org/quotes.html?quote=' + ticker
req = requests.get(url)
soup_oic = BeautifulSoup(req.text, "lxml")

with open("oic_page.html","w", encoding="UTF-8") as fp:
    fp.write(req.text)
    
for iframe in soup_oic.find_all('iframe'):
    if 'ivolatility' in iframe['src']:
        address = iframe['src']
        print(address)
        
req = requests.get(address)
table = BeautifulSoup(req.text, "lxml")
with open("table.html","w", encoding="UTF-8") as fp:
    fp.write(req.text)

https://oic.ivolatility.com/oic_adv_options.j?cnt=a4fe7416d6719b9738e0600c9dd56ba2af2bf4ed41f31ca7&ticker=ON


But still it doesn't work, now the optionseducation.org reports an error. Looking into the inspector it seems that the server detected that we have access not using a browser and qualifies us as a 'robot':
```
<meta name="robots" content="noindex,nofollow">
...
<div id="error500">
    <p class="alert">We're sorry, there was a problem processing your request (error 500)</p>
    <p>Please try again in a few moments and if you continue to have problems, please contact Investor Services at 1-888-OPTIONS.
</p>
</div>
```
How is possible that the server detected that we are accessing to the page through python instead of a browser? The answer could be the headers

If we access to the page <http://www.reliply.org/tools/requestheaders.php> we can see the headers used by our browser to access to this page, in this case:
<table class="standard" cellspacing="1" border="0" cellpadding="4"><tbody><tr><th class="standard" colspan="3"><center><font size="+1"><b>HTTP Request Headers Sent From Your Browser, Plus Explaination</b></font></center></th></tr><tr><td class="standard_evenrow"><center><b>Header Name</b></center></td><td class="standard_evenrow"><center><b><i>Your</i> Header Value</b></center></td><td class="standard_evenrow"><center><b>Explaination</b></center></td></tr><tr><td class="standard_oddrow">Accept Language</td><td class="standard_oddrow">es-ES,es;q=0.9,fr;q=0.8,it;q=0.7</td><td class="standard_oddrow">This is the preferred language of the browser.  Some websites look at this header and change the language of the page according to the accept language.  In many browsers you can change the HTTP Accept Language if you look deep enough in the configuration/settins/options.  Look here for <a href="../info/internet/http-accept-lang.html" title="How To Change My HTTP Accept Language">How To Change My HTTP Accept Language</a></td></tr><tr><td class="standard_evenrow">Accept Encoding</td><td class="standard_evenrow">gzip, deflate</td><td class="standard_evenrow">This tells the webserver if your browser is able to accept compressed webpages.  This can make a big difference in download time and bandwidth usage.  For compressed webpages to be used, the browser must accept them <i>and</i> the webserver must send them.  Look here for an easy way to <a href="../info/internt/enable-compressed-webpages.html" title="Enable Gzip Compression On Your Php Pages To Save Bandwith">Enable Gzip Compression on your Php Webpages</a> (Saves bandwidth and reduces download times).</td></tr><tr><td class="standard_oddrow">User Agent</td><td class="standard_oddrow">Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36</td><td class="standard_oddrow">This identifies the make and model of your browser.  Some websites use this information to indirectly know the capabilities of your browser and change (or tailor) the webpage for your best browsing experience.<br>Some browsers actually allow you to modify the User Agent.  For example the Opera browser Allows you to change its identity to Microsoft Internet Explorer or Mozilla.</td></tr><tr><td class="standard_evenrow">HTTP Connection</td><td class="standard_evenrow"><i>Your browser did not send this request header</i></td><td class="standard_evenrow">Defines the type of connection your browser wants to establish with the webserver.</td></tr><tr><td class="standard_oddrow">Host</td><td class="standard_oddrow">www.reliply.org</td><td class="standard_oddrow">This is the host or domain name that the browser is attempting to connect to.</td></tr><tr><td class="standard_oddrow">Request Method</td><td class="standard_oddrow">GET</td><td class="standard_oddrow">This can be GET, POST, PUT or HEAD.  GET is the "normal" method for viewing a webpage. POST can be used to send form data and the HEAD request method is for retrieving only the headers of a page instead of downloading the entire page.</td></tr></tbody></table>

So if we access the same page but using the requests package of python and save the page to the headers.html file:

In [44]:
import requests
from bs4 import BeautifulSoup

req = requests.get('http://www.reliply.org/tools/requestheaders.php')
soup_headers = BeautifulSoup(req.text, "lxml")

with open("headers.html","w", encoding="UTF-8") as fp:
    fp.write(req.text)

Opening the file:
<table class="standard" cellspacing="1" border="0" cellpadding="4"><tbody><tr><th class="standard" colspan="3"><center><font size="+1"><b>HTTP Request Headers Sent From Your Browser, Plus Explaination</b></font></center></th></tr><tr><td class="standard_evenrow"><center><b>Header Name</b></center></td><td class="standard_evenrow"><center><b><i>Your</i> Header Value</b></center></td><td class="standard_evenrow"><center><b>Explaination</b></center></td></tr><tr><td class="standard_oddrow">Accept Language</td><td class="standard_oddrow"><i>Your browser did not send this request header</i></td><td class="standard_oddrow">This is the preferred language of the browser.  Some websites look at this header and change the language of the page according to the accept language.  In many browsers you can change the HTTP Accept Language if you look deep enough in the configuration/settins/options.  Look here for <a href="../info/internet/http-accept-lang.html" title="How To Change My HTTP Accept Language">How To Change My HTTP Accept Language</a></td></tr><tr><td class="standard_evenrow">Accept Encoding</td><td class="standard_evenrow">gzip, deflate</td><td class="standard_evenrow">This tells the webserver if your browser is able to accept compressed webpages.  This can make a big difference in download time and bandwidth usage.  For compressed webpages to be used, the browser must accept them <i>and</i> the webserver must send them.  Look here for an easy way to <a href="../info/internt/enable-compressed-webpages.html" title="Enable Gzip Compression On Your Php Pages To Save Bandwith">Enable Gzip Compression on your Php Webpages</a> (Saves bandwidth and reduces download times).</td></tr><tr><td class="standard_oddrow">User Agent</td><td class="standard_oddrow">python-requests/2.14.2</td><td class="standard_oddrow">This identifies the make and model of your browser.  Some websites use this information to indirectly know the capabilities of your browser and change (or tailor) the webpage for your best browsing experience.<br>Some browsers actually allow you to modify the User Agent.  For example the Opera browser Allows you to change its identity to Microsoft Internet Explorer or Mozilla.</td></tr><tr><td class="standard_evenrow">HTTP Connection</td><td class="standard_evenrow"><i>Your browser did not send this request header</i></td><td class="standard_evenrow">Defines the type of connection your browser wants to establish with the webserver.</td></tr><tr><td class="standard_oddrow">Host</td><td class="standard_oddrow">www.reliply.org</td><td class="standard_oddrow">This is the host or domain name that the browser is attempting to connect to.</td></tr><tr><td class="standard_oddrow">Request Method</td><td class="standard_oddrow">GET</td><td class="standard_oddrow">This can be GET, POST, PUT or HEAD.  GET is the "normal" method for viewing a webpage. POST can be used to send form data and the HEAD request method is for retrieving only the headers of a page instead of downloading the entire page.</td></tr></tbody></table>

Ahaaa! Comparing  the browser headers, we can see there are two headers that are different:
- Accept language
- User Agent

It seems that the important one is the User Agent, when accessing through Chrome and python, the value is:

| From   | User Agent value  |
|:---|:---:|
| Chrome | Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36 |
| Python | python-requests/2.14.2 |

Likely the server detects this header and deduces that we are accesing through a robot. So the point is to change the header User Agent and use the same as Chrome.

Since it's so easy to do that, let's change the value of Accept-Language as well:

In [45]:
import requests
from bs4 import BeautifulSoup

ticker = 'HPQ'
url = 'https://www.optionseducation.org/quotes.html?quote=' + ticker
headers = {
    'Accept-Language': 'es-ES,es;q=0.9,fr;q=0.8,it;q=0.7',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
}
req = requests.get(url, headers=headers)
soup_oic = BeautifulSoup(req.text, "lxml")

with open("oic_page.html","w", encoding="UTF-8") as fp:
    fp.write(req.text)
    
for iframe in soup_oic.find_all('iframe'):
    if 'ivolatility' in iframe['src']:
        address = iframe['src']
        print(address)
        
req = requests.get(address)
table = BeautifulSoup(req.text, "lxml")
with open("table.html","w", encoding="UTF-8") as fp:
    fp.write(req.text)

https://oic.ivolatility.com/oic_adv_options.j?cnt=a4fe7416d6719b97348d213e597fdafad34ce92e7979084f&ticker=HPQ


Now it works like a piece of cake !!

## Getting the underlying price

We can see in the html structure that the underlying price is actually the first column of the second row of one of the tables. 

Since price is just a float and there are a lot of floats in the tables, we must to search first for a positioning keyword. The keyword string selected is 'Price', we know the price number is the row inmediately below 'Price' string and there is not any other 'Price' string before this. So 'Price' keyword is a perfect selection.

Therefore first we have to iterate over all the tags of the document, to do that we will use:
```
for descendant in soup.descendants:
```
When we detect that one tag is a table:
```
if descendant.name == "table":
```
then again we will iterate over all descendants of that table, looking for rows.

We repeat the process for the rows but now looking for columns, this time we don't need to check all descendants, just the children of each row:
```
for column in row.children:
```

then when a column value is our keyword then we know we have been positioned in the right location, the comparison code is:

```
if (column.get_text()) == 'Price':
```

when this happens, the boolean keywordIsFound gets marked as True and a break is executed to stop looking for more columns in the first row. Now the external iteration, the one that iterates over all rows of the table, loads the next row (the one that we are interested for) and assign the output variable underlying_price to the text value of the first column of that row:
```
underlying_price = row.td.get_text()
```
Finally a sequence of breaks happen and the code print out the underlying price. Awesome

In [47]:
from bs4 import BeautifulSoup

with open("table.html", encoding="UTF-8") as fp:
    text = fp.read()
soup = BeautifulSoup(text, "lxml")
underlying_price = ""
keywordIsFound = False
for descendant in soup.descendants: # look all tags in document
    if descendant.name == "table":  # if one tag is a table
        for row in descendant.descendants:  # look all the tags under it
            if row.name == 'tr': # if one tag is a table row
                if keywordIsFound:
                    underlying_price = row.td.get_text()
                    break
                for column in row.children: # loop all the children tags of the row
                    if column.name == 'td': # if a children tag is a table column
                        if (column.get_text()) == 'Price': # detects the key word, we know the price is next row first column
                            keywordIsFound = True
                            break # stop looking at the columns of the row
    if keywordIsFound:
        break
print('Underlying price = {}'.format(underlying_price))

Underlying price = 21.91


## Code until now

In [55]:
import requests
from bs4 import BeautifulSoup

ticker = 'SPY'
url = 'https://www.optionseducation.org/quotes.html?quote=' + ticker
headers = {
    'Accept-Language': 'es-ES,es;q=0.9,fr;q=0.8,it;q=0.7',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
}
req = requests.get(url, headers=headers)
soup_oic = BeautifulSoup(req.text, "lxml")

with open("oic_page.html","w", encoding="UTF-8") as fp:
    fp.write(req.text)
    
for iframe in soup_oic.find_all('iframe'):
    if 'ivolatility' in iframe['src']:
        address = iframe['src']       
req = requests.get(address)
soup = BeautifulSoup(req.text, "lxml")
with open("table.html","w", encoding="UTF-8") as fp:
    fp.write(req.text)
underlying_price = ""
keywordIsFound = False
for descendant in soup.descendants: # look all tags in document
    if descendant.name == "table":  # if one tag is a table
        for row in descendant.descendants:  # look all the tags under it
            if row.name == 'tr': # if one tag is a table row
                if keywordIsFound:
                    underlying_price = row.td.get_text()
                    break
                for column in row.children: # loop all the children tags of the row
                    if column.name == 'td': # if a children tag is a table column
                        if (column.get_text()) == 'Price': # detects the key word, we know the price is next row first column
                            keywordIsFound = True
                            break # stop looking at the columns of the row
    if keywordIsFound:
        break
print('{0} Underlying Price = {1}'.format(ticker, underlying_price))

SPY Underlying Price = 34.18


However, playing just a bit, I realize that the server continues detecting the access to the page from the script and not by a human, so it got me banned another time.

This could happen for several reasons, for example, it's very suspicious that we always access with the same origin IP and/or using the same (fake) browser. The code in the servers to avoid scrapping from robots could be a little sofisticated.

There are two possible mechanisms we could apply to the script in order to avoid detection as a robot:
- rotate proxies and IP addresses
- fake and rotate user agents

## Getting your output/public ip address

This is your ip adress seen in internet when accessing any server. Let's use the ipify web page:
<https://www.ipify.org/>

In [1]:
from requests import get

ip = get('https://api.ipify.org').text
print('My public IP address is: {}'.format(ip))

My public IP address is: 95.23.32.230


Remember that if the server sees that there are frequent access using this address, it can deduce that you are a robot and not a human being using a web browser. To avoid this we could try to use proxies (anonymous proxies) in which the IP origin is hidden and the server only can see the address of the proxy itself

## Using proxies

See the article
[How To Rotate Proxies and IP Addresses using Python 3](https://www.scrapehero.com/how-to-rotate-proxies-and-ip-addresses-using-python-3/). Using the script shown in that article

In [59]:
import requests
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

proxies = get_proxies()
print(proxies)

{'195.208.172.70:8080', '167.99.188.36:3128', '188.170.222.130:8080', '176.119.112.10:53281', '153.122.54.16:8118', '191.252.100.87:3128', '200.54.194.12:53281', '153.122.52.77:8118', '191.209.72.68:53281'}


Now, spanning a bit the proxy search and encapsulating everything in a class:

In [76]:

import requests
from lxml.html import fromstring

class Get_proxies():
    
    proxies_sources = [
    'https://free-proxy-list.net/',
    'https://www.us-proxy.org/',
    'https://free-proxy-list.net/uk-proxy.html',
    'https://free-proxy-list.net/anonymous-proxy.html',
    'https://www.sslproxies.org/'
    ]

    def get_proxies_from_url(self, url):
        response = requests.get(url)
        parser = fromstring(response.text)
        proxies = list()
        for i in parser.xpath('//tbody/tr')[:10]:
            if i.xpath('.//td[7][contains(text(),"yes")]'):
                #Grabbing IP and corresponding PORT
                proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
                proxies.append(proxy)
        return proxies
    
    def get_proxies(self):
        proxies = []
        for proxies_source in self.proxies_sources:
            proxies.extend(self.get_proxies_from_url(proxies_source))
        return set(proxies)

my_proxies = Get_proxies()
print(my_proxies.get_proxies())

{'46.101.11.131:3128', '207.32.30.20:8080', '159.65.234.60:8080', '181.215.114.144:8123', '89.222.217.185:31588', '35.227.54.242:3128', '147.135.210.114:54566', '23.97.215.153:3128', '191.252.100.68:80', '80.211.189.165:3128', '139.255.57.32:8080', '191.252.100.87:3128', '128.199.254.244:3128', '54.233.85.126:3128', '185.126.201.99:8080', '176.53.37.231:8080', '191.252.195.27:80', '103.214.255.118:3128', '218.148.196.156:808', '187.247.80.56:8080', '117.6.161.118:53281', '152.157.119.253:3128', '54.209.135.103:3128', '41.222.57.164:53281', '178.62.108.246:8118'}


this is the list of free proxies we can use. To see how they "change" our IP public adress:

In [91]:

import requests
from itertools import cycle
import traceback

proxies = Get_proxies()
proxy_pool = cycle(proxies.get_proxies())

url = 'https://httpbin.org/ip'
for i in range(1,11):
    #Get a proxy from the pool
    proxy = next(proxy_pool)
    print("Request {}, server {}".format(i,proxy))
    try:
        response = requests.get(url,proxies={"http": proxy, "https": proxy})
        print(response.json())
    except:
        #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
        #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
        print("-----------------> Skipping. Connnection error")


Request 1, server 46.101.11.131:3128
-----------------> Skipping. Connnection error
Request 2, server 185.145.105.83:53281
-----------------> Skipping. Connnection error
Request 3, server 110.77.189.213:62225
{'origin': '110.77.189.213'}
Request 4, server 42.104.84.107:8080
{'origin': '122.183.242.53'}
Request 5, server 35.227.54.242:3128
{'origin': '35.227.54.242'}
Request 6, server 147.135.210.114:54566
{'origin': '147.135.210.114'}
Request 7, server 23.97.215.153:3128
{'origin': '104.45.31.193'}
Request 8, server 54.39.40.100:3128
-----------------> Skipping. Connnection error
Request 9, server 80.211.189.165:3128
{'origin': '80.211.189.165'}
Request 10, server 191.252.100.87:3128
-----------------> Skipping. Connnection error


We can use the code above to cycle among the proxies. Before that, let's fake and rotate the user agents header

## Faking and rotating User Agents

Again, the information is extracted from [How to fake and rotate User Agents using Python 3](https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/)

One easy way is to install and use the package fake-useragent (in jupyter notebook should be installed through pip)

In [78]:
from fake_useragent import UserAgent
import requests
from lxml.html import fromstring



ua = UserAgent()
url = 'https://httpbin.org/user-agent'
for i in range(1,4):
    user_agent = ua.random
    headers = {'User-Agent': user_agent}
    response = requests.get(url,headers=headers)
    print("Request {0}\nUser-Agent Sent:{1}\nUser Agent Recevied by HTTPBin:".format(i,user_agent))
    print(response.content)
    print("-------------------\n\n")
    



Request 1
User-Agent Sent:Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.2309.372 Safari/537.36
User Agent Recevied by HTTPBin:
b'{"user-agent":"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.2309.372 Safari/537.36"}\n'
-------------------


Request 2
User-Agent Sent:Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36
User Agent Recevied by HTTPBin:
b'{"user-agent":"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36"}\n'
-------------------


Request 3
User-Agent Sent:Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36
User Agent Recevied by HTTPBin:
b'{"user-agent":"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36"}\n'
-------------------




## Using proxies and fake user agents at the same time

In [95]:
import requests
from itertools import cycle
import traceback
from lxml.html import fromstring
import time

proxies = Get_proxies()
proxy_pool = cycle(proxies.get_proxies())
url_1 = 'https://httpbin.org/user-agent'
url_2 = 'https://httpbin.org/ip'
for i in range(1,11):
    #Get a proxy from the pool
    proxy = next(proxy_pool)
    user_agent = ua.random
    headers = {'User-Agent': user_agent}
    print("Request {}".format(i))
    try:
        response = requests.get(url_1, proxies={"http": proxy, "https": proxy}, headers=headers)
        print("User-Agent Sent:\n{}\nUser Agent Recevied by HTTPBin:".format(user_agent))
        print(response.content)
        response = requests.get(url_2, proxies={"http": proxy, "https": proxy}, headers=headers)
        print("Public IP:")
        print(response.json())       
        
    except:
        #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
        #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
        print("Skipping. Connnection error, server {}".format(proxy))
    print("-------------------")

Request 1
User-Agent Sent:
Opera/9.80 (X11; Linux i686; U; es-ES) Presto/2.8.131 Version/11.11
User Agent Recevied by HTTPBin:
b'{"user-agent":"Opera/9.80 (X11; Linux i686; U; es-ES) Presto/2.8.131 Version/11.11"}\n'
Public IP:
{'origin': '46.101.11.131'}
-------------------
Request 2
Skipping. Connnection error, server 185.145.105.83:53281
-------------------
Request 3
Skipping. Connnection error, server 110.77.189.213:62225
-------------------
Request 4
Skipping. Connnection error, server 42.104.84.107:8080
-------------------
Request 5
User-Agent Sent:
Mozilla/5.0 (Windows NT 6.1; rv:22.0) Gecko/20130405 Firefox/22.0
User Agent Recevied by HTTPBin:
b'{"user-agent":"Mozilla/5.0 (Windows NT 6.1; rv:22.0) Gecko/20130405 Firefox/22.0"}\n'
Public IP:
{'origin': '35.227.54.242'}
-------------------
Request 6
User-Agent Sent:
Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0) chromeframe/10.0.648.205
User Agent Recevied by HTTPBin:
b'{"user-agent":"Mozilla/5.0 (compatible; MSI

## Final Code
Now everything is ready to start collecting the prices of different, tickers. 

First let's encapsulate the soup processing in one class:

In [2]:
import requests
from bs4 import BeautifulSoup

class Process_soup():
    def get_underlying_price(self, soup):
        underlying_price = ""
        keywordIsFound = False
        for descendant in soup.descendants: # look all tags in document
            if descendant.name == "table":  # if one tag is a table
                for row in descendant.descendants:  # look all the tags under it
                    if row.name == 'tr': # if one tag is a table row
                        if keywordIsFound:
                            underlying_price = row.td.get_text()
                            break
                        for column in row.children: # loop all the children tags of the row
                            if column.name == 'td': # if a children tag is a table column
                                if (column.get_text()) == 'Price': # detects the key word, we know the price is next row first column
                                    keywordIsFound = True
                                    break # stop looking at the columns of the row
            if keywordIsFound:
                break
        return underlying_price
        

Now, do the same for the get_proxies

In [3]:
import requests
from lxml.html import fromstring


class Get_proxies():
    
    proxies_sources = [
    'https://free-proxy-list.net/',
    'https://www.us-proxy.org/',
    'https://free-proxy-list.net/uk-proxy.html',
    'https://free-proxy-list.net/anonymous-proxy.html',
    'https://www.sslproxies.org/'
    ]

    def get_proxies_from_url(self, url):
        response = requests.get(url)
        parser = fromstring(response.text)
        proxies = list()
        for i in parser.xpath('//tbody/tr')[:10]:
            if i.xpath('.//td[7][contains(text(),"yes")]'):
                #Grabbing IP and corresponding PORT
                proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
                proxies.append(proxy)
        return proxies
    
    def get_proxies(self):
        proxies = []
        for proxies_source in self.proxies_sources:
            proxies.extend(self.get_proxies_from_url(proxies_source))
        return set(proxies)

Finally just iterate over all tickers that we want to extract the underlying price (this is just the begining!).

The code is self-explanatory, also I have added code to save the data got from the url to post-process it, for example looking for option prices or other information:

In [16]:
from itertools import cycle
import traceback
from lxml.html import fromstring
import time
from fake_useragent import UserAgent

tickers = ['SPY','HPQ', 'HPE', 'ILG', 'ON', 'T', 'GE', 'M', 'BAC', 'LEG']
proxies = Get_proxies()
user_agents = UserAgent()
proxy_pool = cycle(proxies.get_proxies())
for ticker in tickers:
    url = 'https://www.optionseducation.org/quotes.html?quote=' + ticker
    for i in proxy_pool:
        proxy = next(proxy_pool) # Get a proxy from the pool
        user_agent = user_agents.random   # Get a random user agent
        headers = {'User-Agent': user_agent}
        try:
            time.sleep(3)
            response = requests.get(url, proxies={"http": proxy, "https": proxy}, headers=headers, timeout = 5)
            soup_oic = BeautifulSoup(response.text, "lxml")
            with open("oic_page_" + ticker + ".html","w", encoding="UTF-8") as fp:
                fp.write(response.text)
            adress = ""
            for iframe in soup_oic.find_all('iframe'):
                if 'ivolatility' in iframe['src']:
                    address = iframe['src']
            req = requests.get(address, proxies={"http": proxy, "https": proxy}, headers=headers, timeout = 5)
            soup = BeautifulSoup(req.text, "lxml")
            with open("price_table_" + ticker + ".html","w", encoding="UTF-8") as fp:
                fp.write(req.text)
            ps = Process_soup()
            print('{0} Underlying Price = {1} got with proxy: {2} '.format(ticker,
                                                                           ps.get_underlying_price(soup),
                                                                           proxy))
            print("-------------------")
            break
        except Exception as e:
            #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
            #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
            print("Error: {}".format(e))
            print("Skipping. Connnection error, server {}".format(proxy))
        print("-------------------")
print("-------END---------")

SPY Underlying Price = 272.15 got with proxy: 192.116.142.153:8080 
-------------------
HPQ Underlying Price = 21.91 got with proxy: 128.199.254.244:3128 
-------------------
HPE Underlying Price = 15.52 got with proxy: 91.194.42.51:80 
-------------------
ILG Underlying Price = 34.18 got with proxy: 202.42.142.66:8080 
-------------------
ON Underlying Price = 25.85 got with proxy: 66.63.9.26:3128 
-------------------
T Underlying Price = 32.51 got with proxy: 185.93.3.123:8080 
-------------------
GE Underlying Price = 14.63 got with proxy: 35.227.54.242:3128 
-------------------
M Underlying Price = 34.13 got with proxy: 153.122.53.124:8118 
-------------------
Error: HTTPSConnectionPool(host='www.optionseducation.org', port=443): Max retries exceeded with url: /quotes.html?quote=BAC (Caused by ConnectTimeoutError(<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x0000000006D75F28>, 'Connection to 152.157.119.253 timed out. (connect timeout=5)'))
Skipping. Con